## Dependencies

In [1]:
import numpy as np
import pandas as pd

## Import and Prep Data

In [2]:
# Player data from CSV
stats = pd.read_csv("../resources/game_skater_stats.csv")
stats.columns

Index(['game_id', 'player_id', 'primaryPosition', 'team_id', 'timeOnIce',
       'assists', 'goals', 'shots', 'hits', 'powerPlayGoals',
       'powerPlayAssists', 'penaltyMinutes', 'faceOffWins', 'faceoffTaken',
       'takeaways', 'giveaways', 'shortHandedGoals', 'shortHandedAssists',
       'blocked', 'plusMinus', 'evenTimeOnIce', 'shortHandedTimeOnIce',
       'powerPlayTimeOnIce'],
      dtype='object')

In [3]:
# Group player ids to get total player stats
player_stats_avg = stats.groupby(["player_id","primaryPosition"],as_index=False).mean().fillna(0)
player_stats_sum = stats.groupby("player_id").sum()

In [4]:
X_df = player_stats_avg.drop(columns=["game_id","team_id","player_id","primaryPosition"])

In [5]:
# Establish X, y
X = X_df.values
y = player_stats_avg["primaryPosition"]

## Create Training and Testing sets

In [6]:
# Import Dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [7]:
# Convert Player Positions (C, W, D) to numerical values
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

In [8]:
## CLASSES
# 0 - C (Center)
# 1 - D (Defense)
# 2 - W (Winger)

In [9]:
# Create train and test splits
X_train, X_test, y_uncat_train, y_uncat_test = train_test_split(X, encoded_y, random_state=42)

In [10]:
# Categorize the y
y_train = to_categorical(y_uncat_train)
y_test = to_categorical(y_uncat_test)

## Build a Deep Neural Network

In [11]:
# Import Dependecies
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [12]:
# Create an empty sequential model
model = Sequential()

In [13]:
# Add the first layer where the input dimensions are the columns of the training data
model.add(Dense(100, activation="relu", input_dim=X_train.shape[1]))

In [14]:
# Add a second hidden layer
model.add(Dense(100, activation="relu"))

In [15]:
# Add output layer
model.add(Dense(units=y_train.shape[1], activation="softmax"))

In [16]:
# Compile the model using categorical_crossentropy for the loss function, the adam optimizer,
# and add accuracy to the training metrics
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [17]:
# Use the training data to fit (train) the model
model.fit(X_train, y_train, epochs=100, shuffle=True, verbose=2)

Epoch 1/100
79/79 - 0s - loss: 4.2361 - accuracy: 0.4618
Epoch 2/100
79/79 - 0s - loss: 1.3486 - accuracy: 0.5378
Epoch 3/100
79/79 - 0s - loss: 2.0469 - accuracy: 0.5119
Epoch 4/100
79/79 - 0s - loss: 1.6721 - accuracy: 0.5628
Epoch 5/100
79/79 - 0s - loss: 1.0373 - accuracy: 0.6245
Epoch 6/100
79/79 - 0s - loss: 0.8697 - accuracy: 0.6671
Epoch 7/100
79/79 - 0s - loss: 0.9677 - accuracy: 0.6432
Epoch 8/100
79/79 - 0s - loss: 1.1122 - accuracy: 0.6420
Epoch 9/100
79/79 - 0s - loss: 0.7404 - accuracy: 0.7021
Epoch 10/100
79/79 - 0s - loss: 0.9431 - accuracy: 0.6647
Epoch 11/100
79/79 - 0s - loss: 1.1526 - accuracy: 0.6372
Epoch 12/100
79/79 - 0s - loss: 0.9858 - accuracy: 0.6826
Epoch 13/100
79/79 - 0s - loss: 0.9422 - accuracy: 0.6639
Epoch 14/100
79/79 - 0s - loss: 0.7288 - accuracy: 0.7279
Epoch 15/100
79/79 - 0s - loss: 0.7424 - accuracy: 0.7275
Epoch 16/100
79/79 - 0s - loss: 0.8569 - accuracy: 0.6830
Epoch 17/100
79/79 - 0s - loss: 0.6183 - accuracy: 0.7673
Epoch 18/100
79/79 - 0s

## Evaluate the Model

In [18]:
# Evaluate the model using the training data
model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

27/27 - 0s - loss: 0.3562 - accuracy: 0.8653
Loss: 0.3561771512031555, Accuracy: 0.865315854549408
